### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [26]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

import operator

from scipy.stats import norm

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [27]:
hyperparameterspace = {
    'x0': ["interval", -5, 10],
    'x1': ["interval", -5, 10],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [28]:


BUDGETS = [1 + i * 52 for i in range(19)]

In [29]:
def visualize(points, adaptivity, alternative):
    x_values = []
    y_values = []
    z_values = []

    for i in range(len(points)):    
        x_values.append(points[i].get_coordinates()[0])
        y_values.append(points[i].get_coordinates()[1])
        z_values.append(points[i].get_value())


    fig = plt.figure()
    ax = plt.axes()
    surface = plt.scatter(x_values, y_values, c=z_values, cmap="plasma")
    plt.scatter(points[0].get_coordinates()[0], points[0].get_coordinates()[1], c="white", marker="x")


    ax.set_xlim([-5, 10])
    ax.set_ylim([-5, 10])

    plt.colorbar(surface)
    plt.gca().set_aspect("equal")

    # plt.savefig("./Alternative_"+str(alternative)+"_adapt_"+str(adaptivity)+".pgf",bbox_inches='tight' )

    plt.show()

### Optimization

In [30]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    sum = 0
    sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    return sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    
    return len(hyperparameterspace) * 10 + sum

def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1])


BUDGET = 200
INIT_POINTS = 5

ADAPTIVITIES = [0.0, 0.75, 1.0]

for adaptivity in ADAPTIVITIES:

    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=0)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))
    print(adaptivity,"best point:", points[0].get_coordinates(), "value", points[0].get_value())


    visualize(points, adaptivity, 0)

    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=1)
    points = optimizer.fit()
    points.sort(key=operator.attrgetter('value'))
    print("best point:", points[0].get_coordinates(), "value", points[0].get_value())

    visualize(points, adaptivity, 1)

    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=2)
    points = optimizer.fit()
    points.sort(key=operator.attrgetter('value'))
    print("best point:", points[0].get_coordinates(), "value", points[0].get_value())

    visualize(points, adaptivity, 2)




0.0 best point: [2.158598821574482, 4.617743382926759] value 1.517121125222536
best point: [0.9541974556509123, 0.9027712932595593] value 0.008060015566753589
best point: [1.097943243714717, 1.1936061770810482] value 0.02369014149435738
0.75 best point: [1.038927050174919, 1.079563922856002] value 0.001519098543161749


/tmp/ipykernel_41889/1183962981.py:26: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


best point: [1.6268172463461463, 2.6253824302103714] value 0.43764024412597213
best point: [0.9605735775731732, 0.838958015201549] value 0.7028532076339621
1.0 best point: [0.4058562472117627, -0.04054645111546973] value 4.566409386159776
best point: [-0.11352135098080107, -0.7217782890446469] value 55.21325276318493
best point: [2.2818252596693043, 4.712738873843234] value 26.04545502357817
